In [1]:
import tensorflow as tf

In [2]:
import numpy as np
import cv2
import math
from PIL import Image, ImageStat
import sys
from os import listdir

In [19]:
def convert(dir):
    file_list=listdir(dir)
    for filename in file_list:
        if "panorama" in filename and "cropped" not in filename:
            path=""
            path=dir+filename
            im=Image.open(path)
            out=im.resize((104,208),Image.ANTIALIAS)
            print ("%s has been resized"%filename)
            out.save(path)

In [20]:
path1 = r"C:\\Users\\35252\\Desktop\\class\\"
files= listdir(path1)

convert(path1)

bigS-0790_-37.80879980770527_144.98194564890238_panorama.png has been resized
bigS-0820_-37.80878714139091_144.98228676055362_panorama.png has been resized
bigS-0860_-37.80877025144173_144.98274157590654_panorama.png has been resized
bigS-1060_-37.80868577546702_144.9850156495497_panorama.png has been resized
bigS-2290_-37.80816528711204_144.99900108792806_panorama.png has been resized
blackC-0850_-37.808774474092964_144.98262787208782_panorama.png has been resized
blackC-0970_-37.80872379506533_144.98399231705417_panorama.png has been resized
blackC-1010_-37.808706898558945_144.98444713162672_panorama.png has been resized
blackC-1630_-37.80844477911086_144.99149673087072_panorama.png has been resized
blackC-2520_-37.8080677765338_145.00161622930622_panorama.png has been resized
buliding-0140_-37.80907400299053_144.9745548677395_panorama.png has been resized
buliding-0380_-37.80897281542424_144.97728377794476_panorama.png has been resized
buliding-1760_-37.808389765304874_144.992974866

In [5]:
def get_image(image_path):
    """
    Get a numpy 2D array of an image so that one can access RGBA[x][y] or RGBA[x,y].
    
    Also get a cropped image for further use for hsv and k-means clustering
    
    """
    origin = Image.open(image_path, 'r')
    width, height = origin.size
    area = (0, 0, width, 0.5*height)
    image = origin.crop(area) # crop top half of the image
#     result = Image.fromarray(image.astype(np.uint8))
    image.save(file+"_cropped.png")
    width, height = image.size
#     print (image.size)
    pixel_values = list(image.getdata())
    if image.mode == 'RGBA':
        channels = 4
    elif image.mode == 'L':
        channels = 1
    else:
        print("Unknown mode: %s" % image.mode)
        return None
    pixel_values = np.array(pixel_values).reshape((height, width, channels))

    return pixel_values #return pixels matrix with RGBA value, and cropped half image

In [6]:
def RGB_array(pixels):
    """
    Get a numpy 2D array of an image so that one can access brightness[x][y]
       
    """
    rgb=[[0 for x in range(len(pixels[0]))] for y in range(len(pixels))]
    # Matrix = [[0 for x in range(w)] for y in range(h)]
    for i in range(len(pixels)):
        for j in range(len(pixels[0])):
            R,G,B,A=pixels[i,j]
            rgb[i][j]=[R,G,B]
    return rgb

In [7]:
train_x = []
for file in sorted(files):
    if "panorama" in file and "cropped" not in file:
#         file=path1+file
        pixels =get_image(file)
#         print (np.shape(pixels))
            # print (pixel1)
        image_rgb = RGB_array(pixels)
        
#             edge_cal(file)
        image_rgb=np.asarray(image_rgb)
        print(np.shape(image_rgb))
        image_rgb=image_rgb.reshape((10816,3))
        train_x.append(image_rgb)
        print(file,np.shape(train_x))

(104, 104, 3)
bigS-0790_-37.80879980770527_144.98194564890238_panorama.png (1, 10816, 3)
(104, 104, 3)
bigS-0820_-37.80878714139091_144.98228676055362_panorama.png (2, 10816, 3)
(104, 104, 3)
bigS-0860_-37.80877025144173_144.98274157590654_panorama.png (3, 10816, 3)
(104, 104, 3)
bigS-1060_-37.80868577546702_144.9850156495497_panorama.png (4, 10816, 3)
(104, 104, 3)
bigS-2290_-37.80816528711204_144.99900108792806_panorama.png (5, 10816, 3)
(104, 104, 3)
blackC-0850_-37.808774474092964_144.98262787208782_panorama.png (6, 10816, 3)
(104, 104, 3)
blackC-0970_-37.80872379506533_144.98399231705417_panorama.png (7, 10816, 3)
(104, 104, 3)
blackC-1010_-37.808706898558945_144.98444713162672_panorama.png (8, 10816, 3)
(104, 104, 3)
blackC-1630_-37.80844477911086_144.99149673087072_panorama.png (9, 10816, 3)
(104, 104, 3)
blackC-2520_-37.8080677765338_145.00161622930622_panorama.png (10, 10816, 3)
(104, 104, 3)
buliding-0140_-37.80907400299053_144.9745548677395_panorama.png (11, 10816, 3)
(104, 

In [8]:
train_y = []
for i in range(5):
    train_y.append([1,0,0,0,0,0])
for i in range(5):
    train_y.append([0,1,0,0,0,0])
for i in range(3):
    train_y.append([0,0,1,0,0,0])
for i in range(5):
    train_y.append([0,0,0,1,0,0])
for i in range(5):
    train_y.append([0,0,0,0,1,0])
for i in range(5):
    train_y.append([0,0,0,0,0,1])
train_y = np.array(train_y)
print(np.shape(train_y))

(28, 6)


In [9]:
test_x = train_x[0]
test_y = train_y[0]
train_x=np.delete(train_x,1,0)
train_y=np.delete(train_y,1,0)
print(np.shape(train_x),np.shape(train_y))

(27, 10816, 3) (27, 6)


In [10]:
test_x=test_x.reshape((1,10816, 3))
print(np.shape(test_x))

(1, 10816, 3)


In [11]:
test_y= np.array(test_y)
test_y=test_y.reshape((1,-1))
print(np.shape(test_y))

(1, 6)


In [12]:
# init weight
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev = 0.1) #normal distribution
    return tf.Variable(initial)

In [13]:
# init bias
def bias_variable(shape):
    initial = tf.constant(0.1, shape = shape)
    return tf.Variable(initial)

In [14]:
def conv2d(x,W):
#     x input tensor of shape [batch, in_height, in_width, in_channels]
# W filter/kernel tensor of shape [filter_height, filter_width, in_channels, out_channels]
# strides[0]=strides[3]=1 strides[x] stands "x"direction, strides[y]stands "y" dicrection
# padding: "SAME",OR "VALID"
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

In [15]:
def max_pool_2x2(x):
#     ksize [1,x,y,1]
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")

In [16]:
# batch_size = 100

# n_batch = mnist.train.num_examples // batch_size

batch_size=27

#placeholder
x = tf.placeholder(tf.float32,[None,10816,3])#104x104
y = tf.placeholder(tf.float32,[None,6])

# change X format into a 4D vector [batch, in_height, in_width, in_channels]
x_image = tf.reshape(x,[-1,104,104,3])

""""""
# init first Convolutional Layer
W_conv1 = weight_variable([5,5,3,32]) #5*5 window, 32 kernals extract feature from 3
b_conv1 = bias_variable([32]) #each kernal 1 bias

# Convolution using relu activation function
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1) +b_conv1)
h_pool1 = max_pool_2x2(h_conv1)  #max-pooling

""""""
# init second Convolutional Layer
W_conv2 = weight_variable([5,5,32,64]) #5*5 window, 64 kernals extract feature from 3
b_conv2 = bias_variable([64]) #each kernal 1 bias

# Convolution using relu activation function
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2) +b_conv2)
h_pool2 = max_pool_2x2(h_conv2)  #max-pooling

# 104*104 image → 52*52 after first pooling
# 52*52 image → 26*26 after second pooling
# get 64*26*26

In [17]:
# init FC layer
W_fc1 = weight_variable([64*26*26,1024]) #last layer 64*26*26 neurons
b_fc1 = bias_variable([1024])

# conver output of second pooling layer into 1 demonsion
h_pool2_flat= tf.reshape(h_pool2, [-1,64*26*26])

# first FC layer output
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)

temp_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1,temp_prob)

""""""

# second FC layer
W_fc2 = weight_variable([1024,6])
b_fc2 = bias_variable([6])

""""""
# output
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2)+b_fc2)

In [18]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits = prediction))

# AdamOptimizer
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# store into a boolean list
correct_prediction=  tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))#argmax return the position of the max number

# get accuracy
accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(21):
        for batch in range(batch_size):
            batch_xs,batch_ys =  train_x,train_y
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys, temp_prob:0.7 })
        
        acc = sess.run(accuracy,feed_dict={x:test_x,y:test_y, temp_prob:1.0 })
        print("Iter " + str(epoch) + ",Testing Accuracy " + str(acc))


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



KeyboardInterrupt: 